In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Quantum detection of time series anomalies
==========================================

::: {.meta}
:property=\"og:description\": Learn how to quantumly detect anomalous
behaviour in time series data with the help of Covalent.
:property=\"og:image\":
<https://pennylane.ai/qml/_images/thumbnail_tutorial_univariate_qvr.jpg>
:::

::: {.related}
tutorial\_qaoa\_intro Intro to QAOA
:::

*Authors: Jack Stephen Baker, Santosh Kumar Radha --- Posted: 7 February
2023.*

Systems producing observable characteristics which evolve with time are
almost everywhere we look. The temperature changes as day turns to
night, stock markets fluctuate and the bacteria colony living in the
coffee cup to your right, which you *promised* you would clean
yesterday, is slowly growing (seriously, clean it). In many situations,
it is important to know when these systems start behaving abnormally.
For example, if the pressure inside a nuclear fission reactor starts
violently fluctuating, you may wish to be alerted of that. The task of
identifying such temporally abnormal behaviour is known as time series
anomaly detection and is well known in machine learning circles.

In this tutorial, we take a stab at time series anomaly detection using
the *Quantum Variational Rewinding* algorithm, or QVR, proposed by
[Baker, Horowitz, Radha et. al (2022)](https://arxiv.org/abs/2210.16438)
--- a quantum machine learning algorithm for gate model quantum
computers. QVR leverages the power of unitary time evolution/devolution
operators to learn a model of *normal* behaviour for time series data.
Given a new (i.e., unseen in training) time series, the normal model
produces a value that, beyond a threshold, defines anomalous behaviour.
In this tutorial, we'll be showing you how all of this works, combining
elements from [Covalent](https://www.covalent.xyz/),
[Pennylane](https://pennylane.ai/) and [PyTorch](https://pytorch.org/).

Before getting into the technical details of the algorithm, let\'s get a
high-level overview with the help of the cartoon below.


![](../demonstrations/univariate_qvr/cartoon_pennylane.png){.align-center
width="70.0%"}

Going left-to-right, a time series is sampled at three points in time,
corresponding to different stages in the life cycle of a butterfly: a
catepillar, a chrysalis and a butterfly. This information is then
encoded into quantum states and passed to a time machine which time
devolves the states as generated by a learnt Hamiltonian operator (in
practice, there is a distribution of such operators). After the devolved
state is measured, the time series is recognized as normal if the
average measurement is smaller than a given threshold and anomalous if
the threshold is exceeded. In the first case, the time series is
considered rewindable, correctly recovering the initial condition for
the life cycle of a butterfly: eggs on a leaf. In the second case, the
output is unrecognizable.

This will all make more sense once we delve into the math a little.
Let\'s do it!


Background
==========

To begin, let's quickly recount the data that QVR handles: time series.
A general time series $\boldsymbol{y}$ can be described as a sequence of
$p$-many observations of a process/system arranged in chronological
order, where $p$ is a positive integer:

$$\boldsymbol{y} := (\boldsymbol{y}_t: t \in T), \quad T := (t_l: l \in \mathbb{Z}^{+}_{\leq p}).$$

In the simple and didactic case treated in this tutorial,
$\boldsymbol{y}$ is univariate (i.e, is a one-dimensional time series),
so bold-face for $\boldsymbol{y}$ is dropped from this point onwards.
Also, we take $y_t \in \mathbb{R}$ and $t_l \in \mathbb{R}_{>0}$.

The goal of QVR and many other (classical) machine learning algorithms
for time series anomaly detection is to determine a suitable *anomaly
score* function $a_{X}$, where $X$ is a training dataset of *normal*
time series instances $x \in X$ ($x$ is defined analogously to $y$ in
the above), from which the anomaly score function was learnt. When
passed a general time series $y$, this function produces a real number:
$a_X(y) \in \mathbb{R}$. The goal is to have $a_X(x) \approx 0$, for all
$x \in X$. Then, for an unseen time series $y$ and a threshold
$\zeta \in \mathbb{R}$, the series is said to be anomalous should
$a_X(y) > \zeta,$ and normal otherwise. We show a strategy for setting
$\zeta$ later in this tutorial.

The first step for doing all of this *quantumly* is to generate a
sequence $\mathcal{S} := (|x_{t} \rangle: t \in T)$ of $n$-qubit quantum
states corresponding to a classical time series instance in the training
set. Now, we suppose that each $|x_t \rangle$ is a quantum state evolved
to a time $t$, as generated by an *unknown embedding Hamiltonian* $H_E$.
That is, each element of $\mathcal{S}$ is defined by
$|x_t \rangle = e^{-iH_E(x_t)}|0\rangle^{\otimes n} = U(x_t)|0\rangle^{\otimes n}$
for an embedding unitary operator $U(x_t)$ implementing a quantum
feature map (see the [Pennylane embedding
templates](https://docs.pennylane.ai/en/stable/introduction/templates.html#embedding-templates)
for efficient quantum circuits for doing so). Next, we operate on each
$|x_t\rangle$ with a parameterized
$e^{-iH(\boldsymbol{\alpha}, \boldsymbol{\gamma})t}$ operator to prepare
the states

$$|x_t, \boldsymbol{\alpha}, \boldsymbol{\gamma}\rangle := e^{-iH(\boldsymbol{\alpha}, \boldsymbol{\gamma})t}|x_t\rangle,$$

where we write $e^{-iH(\boldsymbol{\alpha}, \boldsymbol{\gamma})t}$ as
an eigendecomposition

$$V_t(\boldsymbol{\alpha}, \boldsymbol{\gamma}) := W^{\dagger}(\boldsymbol{\alpha})D(\boldsymbol{\gamma}, t)W(\boldsymbol{\alpha}) = e^{-iH(\boldsymbol{\alpha}, \boldsymbol{\gamma})t}.$$

Here, the unitary matrix of eigenvectors $W(\boldsymbol{\alpha})$ is
parametrized by $\boldsymbol{\alpha}$ and the unitary diagonalization
$D(\boldsymbol{\gamma}, t)$ is parametrized by $\boldsymbol{\gamma}.$
Both can be implemented efficiently using parameterized quantum
circuits. The above equality with
$e^{-iH(\boldsymbol{\alpha}, \boldsymbol{\gamma})t}$ is a consequence of
Stone's theorem for strongly continuous one-parameter unitary groups.

We now ask the question: *What condition is required for*
$|x_t, \boldsymbol{\alpha}, \boldsymbol{\gamma} \rangle = |0 \rangle^{\otimes n}$
*for all time?* To answer this, we impose
$P(|0\rangle^{\otimes n}) = |\langle 0|^{\otimes n}|x_t, \boldsymbol{\alpha}, \boldsymbol{\gamma} \rangle|^2 = 1.$
Playing with the algebra a little, we find that the following condition
must be satisfied for all $t$:

$$\langle 0|^{\otimes n}e^{-iH(\boldsymbol{\alpha}, \boldsymbol{\gamma})t}e^{-iH_E(x_t)}|0\rangle^{\otimes n} = 1 \iff  H(\boldsymbol{\alpha}, \boldsymbol{\gamma})t = -H_E(x_t).$$

In other words, for the above to be true, the parameterized unitary
operator $V_t(\boldsymbol{\alpha}, \boldsymbol{\gamma})$ should be able
to reverse or *rewind* $|x_t\rangle$ to its initial state
$|0\rangle^{\otimes n}$ before the embedding unitary operator $U(x_t)$
was applied.

We are nearly there! Because it is reasonable to expect that a single
Hamiltonian will not be able to successfully rewind every $x \in X$ (in
fact, this is impossible to do if each $x$ is unique, which is usually
true), we consider the average effect of many Hamiltonians generated by
drawing $\boldsymbol{\gamma}$ from a normal distribution
$\mathcal{N}(\mu, \sigma)$ with mean $\mu$ and standard deviation
$\sigma$:

$$F(\boldsymbol{\phi}, x_t) := \mathop{\mathbb{E}_{\boldsymbol{\gamma} \sim \mathcal{N}(\mu, \sigma)}}\left[\langle 0|^{\otimes n} |x_t, \boldsymbol{\alpha}, \boldsymbol{\gamma}\rangle  \right], \quad \boldsymbol{\phi} = [\boldsymbol{\alpha}, \mu, \sigma].$$

The goal is for the function $F$ defined above to be as close to $1$ as
possible, for all $x \in X$ and $t \in T.$ With this in mind, we can
define the loss function to minimize as the mean square error
regularized by a penalty function $P_{\tau}(\sigma)$ with a single
hyperparameter $\tau$:

$$\mathcal{L(\boldsymbol{\phi})} = \frac{1}{2|X||T|}\sum_{x \in X} \sum_{t \in T}[1 - F(\boldsymbol{\phi}, x_t)]^2 + P_{\tau}(\sigma).$$

We will show the exact form of $P_{\tau}(\sigma)$ later. The general
purpose of the penalty function is to penalize large values of $\sigma$
(justification for this is given in the Supplement of). After
approximately finding the argument $\boldsymbol{\phi}^{\star}$ that
minimizes the loss function (found using a classical optimization
routine), we finally arrive at a definition for our anomaly score
function $a_X(y)$

$$a_X(y) = \frac{1}{|T|}\sum_{t \in T}[1 - F(\boldsymbol{\phi}^{\star}, y_t)]^2.$$

It may now be apparent that we have implemented a clustering algorithm!
That is, our model $F$ was trained such that normal time series
$x \in X$ produce $F(\boldsymbol{\phi}^{\star}, x_t)$ clustered about a
center at $1$. Given a new time series $y$, should
$F(\boldsymbol{\phi}^{\star}, y_t)$ venture far from the normal center
at $1$, we are observing anomalous behaviour!

Take the time now to have another look at the cartoon at the start of
this tutorial. Hopefully things should start making sense now.

Now with our algorithm defined, let's stitch this all together: enter
[Covalent](https://www.covalent.xyz/).


Covalent: heterogeneous workflow orchestration
==============================================

Presently, many QML algorithms are *heterogeneous* in nature. This means
that they require computational resources from both classical and
quantum computing. Covalent is a tool that can be used to manage their
interaction by sending different tasks to different computational
resources and stitching them together as a workflow. While you will be
introduced to other concepts in Covalent throughout this tutorial, we
define two key components to begin with.

1.  **Electrons**. Decorate regular Python functions with `@ct.electron`
    to desginate a *task*. These are the atoms of a computation.

2.  **Lattices**. Decorate a regular Python function with `@ct.lattice`
    to designate a *workflow*. These contain electrons stitched together
    to do something useful.

    Different electrons can be run remotely on different hardware and
    multiple computational paridigms (classical, quantum, etc.: see the
    [Covalent
    executors](https://covalent.readthedocs.io/en/stable/plugins.html)).
    In this tutorial, however, to keep things simple, tasks are run on a
    local Dask cluster, which provides (among other things)
    auto-parallelization.

![\| A schematic demonstrating the different platforms Covalent can
interact
with.](../demonstrations/univariate_qvr/covalent_platform.png){.align-center
width="70.0%"}

Now is a good time to import Covalent and launch the Covalent server!


In [ ]:
# Set up Covalent server


# If you run into any out-of-memory issues with Dask when running this notebook,
# Try reducing the number of workers and making a specific memory request. I.e.:
# os.system("covalent start -m "2GiB" -n 2")
# try covalent –help for more info

Generating univariate synthetic time series
===========================================

In this tutorial, we shall deal with a simple and didactic example.
Normal time series instances are chosen to be noisy low-amplitude
signals normally distributed about the origin. In our case,
$x_t \sim \mathcal{N}(0, 0.1)$. Series we deem to be anomalous are the
same but with randomly inserted spikes with random durations and
amplitudes.

Let's make a `@ct.electron` to generate each of these synthetic time
series sets. For this, we\'ll need to import Torch. We\'ll also set the
default tensor type and pick a random seed for the whole tutorial for
reproducibility.


In [ ]:
# Seed Torch for reproducibility and set default tensor type

Let\'s do a quick sanity check and plot a couple of these series.
Despite the above function\'s `@ct.electron` decorators, these can still
be used as normal Python functions without using the Covalent server.
This is useful for quick checks like this:


Taking a look at the above, the generated series are what we wanted. We
have a simple human-parsable notion of what it is for a time series to
be anomalous (big spikes). Of course, we don\'t need a complicated
algorithm to be able to detect such anomalies but this is just a
didactic example remember!

Like many machine learning algorithms, training is done in mini-batches.
Examining the form of the loss function
$\mathcal{L}(\boldsymbol{\phi})$, we can see that time series are
atomized. In other words, each term in the mean square error is for a
given $x_t$ and not measured against the entire series $x$. This allows
us to break down the training set $X$ into time-series-independent
chunks. Here's an electron to do that:


We now wish to pass this to a cycled `torch.utils.data.DataLoader`.
However, this object is not
[pickleable](https://docs.python.org/3/library/pickle.html#:~:text=%E2%80%9CPickling%E2%80%9D%20is%20the%20process%20whereby,back%20into%20an%20object%20hierarchy.),
which is a requirement of electrons in Covalent. We therefore use the
below helper class to create a pickleable version.


We call an instance of the above in an electron


We now have the means to create synthetic data and cycle through a
training set. Next, we need to build our loss function
$\mathcal{L}(\boldsymbol{\phi})$ from electrons with the help of
`PennyLane`.


Building the loss function
==========================

Core to building the loss function is the quantum circuit implementing
$V_t(\boldsymbol{\alpha}, \boldsymbol{\gamma}) := W^{\dagger}(\boldsymbol{\alpha})D(\boldsymbol{\gamma}, t)W(\boldsymbol{\alpha})$.
While there are existing templates in `PennyLane` for implementing
$W(\boldsymbol{\alpha})$, we use a custom circuit to implement
$D(\boldsymbol{\gamma}, t)$. Following the approach taken in (also
explained in and the appendix of), we create the electron:


While the above may seem a little complicated, since we only use a
single qubit in this tutorial, the resulting circuit is merely a single
$R_z(\theta)$ gate.


You may find the general function for $D$\` useful in case you want to
experiment with more qubits and your own (possibly multi-dimensional)
data after this tutorial.

Next, we define a circuit to calculate the probability of certain bit
strings being measured in the computational basis. In our simple
example, we work only with one qubit and use the `default.qubit` local
quantum circuit simulator.


To take the projector $|0\rangle^{\otimes n} \langle 0 |^{\otimes n}$,
we consider only the probability of measuring the bit string of all
zeroes, which is the 0th element of the probabilities (bit strings are
returned in lexicographic order).


We now have the necessary ingredients to build
$F(\boldsymbol{\phi}, x_t)$.


We now return to the matter of the penalty function $P_{\tau}$. We
choose

$$P_{\tau}(\sigma) := \frac{1}{\pi} \arctan(2 \pi \tau |\sigma|).$$

As an electron, we have


The above is a sigmoidal function chosen because it comes with the
useful property of being bounded. The prefactor of $1/\pi$ is chosen
such that the final loss $\mathcal{L}(\boldsymbol{\phi})$ is defined in
the range (0, 1), as defined in the below electron.


Training the normal model
=========================

Now equipped with a loss function, we need to minimize it with a
classical optimization routine. To start this optimization, however, we
need some initial parameters. We can generate them with the below
electron.


Using the `PyTorch` interface to `PennyLane`, we define our final
electron before running the training workflow.


Now, enter our first `@ct.lattice`. This combines the above electrons,
eventually returning the optimal parameters $\boldsymbol{\phi}^{\star}$
and the loss with batch iterations.


Before running this workflow, we define all of the input parameters.


We can now dispatch the lattice to the Covalent server.


If you are running the notebook version of this tutorial, if you
navigate to <http://localhost:48008/> you can view the workflow on the
Covalent GUI. It will look like the screenshot below, showing nicely how
all of the electrons defined above interact with each other in the
workflow. You can also track the progress of the calculation here.

![A screenshot of the Covalent GUI for the training
workflow.](../demonstrations/univariate_qvr/covalent_tutorial_screenshot.png){.align-center
width="85.0%"}


We now pull the results back from the Covalent server:


and take a look at the training loss history:


Tuning the threshold $\zeta$
============================

When we have access to labelled anomalous series (as we do in our toy
problem here, often not the case in reality), we can tune the threshold
$\zeta$ to maximize some success metric. We choose to maximize the
accuracy score as defined using the three electrons below.


Then, knowing the anomaly scores $a_X(y)$ for a validation data set, we
can scan through various values of $\zeta$ on a fine 1D grid and
calcuate the accuracy score. Our goal is to pick the $\zeta$ with the
largest accuracy score.


We now create our second `@ct.lattice`. We are to test the optimal model
against two random models. If our model is trainable, we should see that
the trained model is better.


We now set the input parameters.


As before, we dispatch the lattice to the `Covalent` server.


Now, we can plot the results:


Parsing the above, we can see that the optimal model achieves high
accuracy when the threshold is tuned using the validation data. On the
other hand, the random models return mostly random results (sometimes
even worse than random guesses), regardless of how we set the threshold.


Testing the model
=================

Now with optimal thresholds for our optimized and random models, we can
perform testing. We already have all of the electrons to do this, so we
create the `@ct.lattice`


We dispatch it to the Covalent server with the appropriate parameters.


Finally, we plot the results below.


As can be seen, once more, the trained model is far more accurate than
the random models. Awesome! Now that we\'re done with the calculations
in this tutorial, we just need to remember to shut down the Covalent
server


In [ ]:
# Shut down the covalent server

Conclusions
===========

We\'ve now reached the end of this tutorial! Quickly recounting what we
have learnt, we:

1.  Learnt the background of how to detect anomalous time series
    instances, *quantumly*,
2.  Learnt how to build the code to achieve this using PennyLane and
    PyTorch, and,
3.  Learnt the basics of Covalent: a workflow orchestration tool for
    heterogeneous computation

If you want to learn more about QVR, you should consult the paper where
we generalize the math a little and test the algorithm on less trivial
time series data than was dealt with in this tutorial. We also ran some
experiments on real quantum computers, enhancing our results using error
mitigation techniques. If you want to play some more with Covalent,
check us out on [GitHub](https://github.com/AgnostiqHQ/covalent/) and/or
engage with other tutorials in our
[documentation](https://covalent.readthedocs.io/en/stable/).


References
==========

About the authors \-\-\-\-\-\-\-\-\-\-\-\-\-\-\--.. include::
../\_static/authors/jack\_stephen\_baker.txt .. include::
../\_static/authors/santosh\_kumar\_radha.txt
